In [57]:
import torch
from torch.distributions import Bernoulli
from torch.optim import Adam
from torch.distributions import Beta
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import ipywidgets as widgets
from IPython.display import display


Bernoulli Distribution

In [58]:
p=0.6
dist=Bernoulli(torch.tensor([p]))
dist

Bernoulli(probs: tensor([0.6000]))

In [59]:
sample=dist.sample()
print("Sample:", sample)

Sample: tensor([1.])


In [60]:
prob = (p ** sample) * ((1 - p) ** (1 - sample))
print("Probability:", prob)
log_prob=dist.log_prob(sample)
print(log_prob)

Probability: tensor([0.6000])
tensor([-0.5108])


log(P(x; p)) = x * log(p) + (1 - x) * log(1 - p)


In [61]:
size = 10
dataset = dist.sample(torch.Size([size]))
probs = dist.log_prob(dataset).exp()
for i in range(size):
    sample = dataset[i].item()
    print("Sample:", sample, "Prob:", probs[i].item())
    print("Sample:", sample, "Log Prob:", (p ** sample) * ((1 - p) ** (1 - sample)))


Sample: 0.0 Prob: 0.3999999761581421
Sample: 0.0 Log Prob: 0.4
Sample: 0.0 Prob: 0.3999999761581421
Sample: 0.0 Log Prob: 0.4
Sample: 1.0 Prob: 0.6000000834465027
Sample: 1.0 Log Prob: 0.6
Sample: 1.0 Prob: 0.6000000834465027
Sample: 1.0 Log Prob: 0.6
Sample: 0.0 Prob: 0.3999999761581421
Sample: 0.0 Log Prob: 0.4
Sample: 1.0 Prob: 0.6000000834465027
Sample: 1.0 Log Prob: 0.6
Sample: 1.0 Prob: 0.6000000834465027
Sample: 1.0 Log Prob: 0.6
Sample: 0.0 Prob: 0.3999999761581421
Sample: 0.0 Log Prob: 0.4
Sample: 1.0 Prob: 0.6000000834465027
Sample: 1.0 Log Prob: 0.6
Sample: 1.0 Prob: 0.6000000834465027
Sample: 1.0 Log Prob: 0.6


Maximum Likelihood Estimations of Bernoulli Distribution

MLE = (number of successes) / (total number of observations)


In [62]:
dataset
num_suc=dataset.float().sum()
p_estimate=num_suc.float()/dataset.size(0)
print("MLE Estimate:", p_estimate.item())

MLE Estimate: 0.6000000238418579


Maximum Likelihood Estimate by Automatic Differentiation

NLL(p; x) = -[x * log(p) + (1 - x) * log(1 - p)]


In [63]:
p = torch.tensor(0.5, requires_grad=True)
def negative_log_likelihood(p):
    return -(dataset * torch.log(p) + (1 - dataset) * torch.log(1 - p)).mean()

optimizer = Adam([p], lr=0.1)

for i in range(100):
    optimizer.zero_grad()
    loss=negative_log_likelihood(p)
    loss.backward()
    optimizer.step()
mle_estimate=p.item()
print("MLE Estimate ", mle_estimate)

MLE Estimate  0.6003569960594177


Beta Distrubution : f(x; α, β) = (1/B(α, β)) * x^(α-1) * (1-x)^(β-1)

In [64]:

alpha = torch.tensor(2.0)
beta = torch.tensor(5.0)
#beta_dist = dist.Beta(alpha, beta)
sample = beta_dist.sample()
print("Sample:", sample.item())
log_prob = beta_dist.log_prob(sample)
print("Log Probability:", log_prob.item())

Sample: 0.3387662470340729
Log Probability: 0.664161205291748


In [65]:

x = np.linspace(0, 1, 1000)
x
def plot(alpha, beta):
    pdf = stats.beta.pdf(x, alpha, beta)
    if np.any(np.isnan(pdf)) or np.any(np.isinf(pdf)):
        print("Invalid parameter values. Please choose different values.")
        return
    plt.clf()
    plt.plot(x, pdf)
    plt.title(f"Alpha = {alpha:.1f}, Beta = {beta:.1f}")
    plt.xlabel('x')
    plt.ylabel('PDF')
    plt.ylim(0, np.max(pdf) * 1.1)
    plt.show()
alpha_slider = widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=0.5, description='Alpha:')
beta_slider = widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=0.5, description='Beta:')
output = widgets.interactive_output(plot, {'alpha': alpha_slider, 'beta': beta_slider})
display(widgets.VBox([alpha_slider, beta_slider, output]))


Posterior for the Bernoulli using the Conjugate Prior
P(p∣k,n,α,β)=Beta(p∣α+k,β+n−k)

In [66]:
prior_alpha = 2
prior_beta = 2
prior_dist = Beta(prior_alpha, prior_beta)
n_ones = dataset.sum()
print(n_ones)
n_zeros = len(dataset) - n_ones
print(n_zeros)
posterior_alpha = prior_alpha + n_ones
posterior_beta = prior_beta + n_zeros
posterior_dist = Beta(posterior_alpha, posterior_beta)
sample = posterior_dist.sample()
print("Sample from Posterior:", sample.item())
log_prob = posterior_dist.log_prob(sample)
print("Log Probability from Posterior:", log_prob.item())
posterior_alpha
posterior_beta

tensor(6.)
tensor(4.)
Sample from Posterior: 0.6189901828765869
Log Probability from Posterior: 1.0571985244750977


tensor(6.)

In [67]:

def update_plot(alpha, beta):
    prior_dist = Beta(alpha, beta)
    n_ones = dataset.sum()
    n_zeros = len(dataset) - n_ones
    posterior_alpha = alpha + n_ones
    posterior_beta = beta + n_zeros
    posterior_dist = Beta(posterior_alpha, posterior_beta)
    log_pdf = posterior_dist.log_prob(torch.tensor(x))
    pdf = torch.exp(log_pdf)
    
    plt.clf()
    
    plt.plot(x, pdf)
    plt.title(f"Posterior Distribution (Alpha = {posterior_alpha}, Beta = {posterior_beta})")
    plt.xlabel('x')
    plt.ylabel('PDF')
    
    plt.show()

alpha_slider = widgets.FloatSlider(min=0, max=10, step=0.1, value=2.0, description='Alpha:')
beta_slider = widgets.FloatSlider(min=0, max=10, step=0.1, value=2.0, description='Beta:')

output = widgets.interactive_output(update_plot, {'alpha': alpha_slider, 'beta': beta_slider})

display(widgets.VBox([alpha_slider, beta_slider, output]))


MAP Estimate = (Number of Successes + Prior Alpha) / (Total Observations + Prior Alpha + Prior Beta)

In [68]:
prior_alpha = 1.0
prior_beta = 1.0
total_observations = len(dataset)
num_successes = dataset.sum().item()
map_estimate = (num_successes + prior_alpha) / (total_observations + prior_alpha + prior_beta)
print("MAP Estimate:", map_estimate)



MAP Estimate: 0.5833333333333334


In [69]:
prior_alpha = 1.0
prior_beta = 1.0
total_observations = len(dataset)
num_successes = dataset.sum().item()
map_estimate = (num_successes + prior_alpha) / (total_observations + prior_alpha + prior_beta)
print("MAP Estimate:", map_estimate)

MAP Estimate: 0.5833333333333334
